In [2]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel, AutoConfig
import torch.optim as optim
from mlx_lm import load, generate
from tqdm import tqdm
import chromadb
from chromadb.utils import embedding_functions
import torch
from chromadb import Documents, EmbeddingFunction, Embeddings
from numpy.typing import ArrayLike
from sentence_transformers.util import cos_sim, dot_score
import numpy as np
import torch.nn as nn
from peft import PeftModel
from llm2vec import LLM2Vec

In [3]:
class Embedding(EmbeddingFunction):
    def __init__(self):
        self.embedding_model = SentenceTransformer('thenlper/gte-large')  # ("Salesforce/SFR-Embedding-Mistral") (7b)

    def encode(self, input: Documents) -> Embeddings:
        if isinstance(input, str):
            input = [input]
        embeddings = self.embedding_model.encode(input)
        return embeddings
    
    def __call__(self, input: Documents) -> Embeddings:
        if isinstance(input, str):
            input = [input]
        embeddings = self.embedding_model.encode(input)
        return embeddings

    def distance(self, x: ArrayLike, y: ArrayLike) -> float:
        return cos_sim(x, y)

In [6]:
class PeftEmbeddingModel(EmbeddingFunction):
    def __init__(self):
        config = AutoConfig.from_pretrained("McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp", trust_remote_code=True)
        tokenizer = AutoTokenizer.from_pretrained("McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp")

        # Loading base MNTP model, along with custom code that enables bidirectional connections in decoder-only LLMs
        tokenizer = AutoTokenizer.from_pretrained(
            "McGill-NLP/LLM2Vec-Mistral-7B-Instruct-v2-mntp"
        )
        config = AutoConfig.from_pretrained(
            "McGill-NLP/LLM2Vec-Mistral-7B-Instruct-v2-mntp", trust_remote_code=True
        )
        model = AutoModel.from_pretrained(
            "McGill-NLP/LLM2Vec-Mistral-7B-Instruct-v2-mntp",
            trust_remote_code=True,
            config=config,
            torch_dtype=torch.float16,
            device_map="mps",
        )
        model = PeftModel.from_pretrained(
            model,
            "McGill-NLP/LLM2Vec-Mistral-7B-Instruct-v2-mntp",
        )
        model = model.merge_and_unload()  # This can take several minutes on cpu

        # Loading unsupervised-trained LoRA weights. This loads the trained LoRA weights on top of MNTP model. Hence the final weights are -- Base model + MNTP (LoRA) + SimCSE (LoRA).
        model = PeftModel.from_pretrained(
            model, "McGill-NLP/LLM2Vec-Mistral-7B-Instruct-v2-mntp-unsup-simcse"
        )

        # Or loading supervised-trained LoRA weights
        model = PeftModel.from_pretrained(
            model, "McGill-NLP/LLM2Vec-Mistral-7B-Instruct-v2-mntp-supervised"
        )
        self.l2v = LLM2Vec(model, tokenizer, pooling_mode="mean", max_length=512)

    def __call__(self, input: Documents) -> Embeddings:
        if isinstance(input, str):
            input = [input]
        embeddings = self.l2v.encode(input)
        return embeddings
    
    def encode(self, input: Documents) -> Embeddings:
        embeddings = self(input)
        return embeddings

In [7]:
peftEmbeddingModel = PeftEmbeddingModel()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of the model checkpoint at mistralai/Mistral-7B-Instruct-v0.2 were not used when initializing MistralEncoderModel: ['lm_head.weight']
- This IS expected if you are initializing MistralEncoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MistralEncoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


TypeError: BFloat16 is not supported on MPS

In [3]:
client = chromadb.PersistentClient(path="./chromadb")
embedding_function = Embedding()

collection = client.get_collection("Python-RAG", embedding_function=embedding_function)

In [4]:
llm_model, tokenizer = load("mlx-community/quantized-gemma-7b-it")

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

In [5]:
data = collection.get()
questions = []
for index in tqdm(range(len(data["ids"])), desc="Generating questions"):
    id = data["ids"][index]
    document = data["documents"][index]
    embeddings = embedding_function.encode(document)[0]
    prompt = f"<start_of_turn>user Create a vague short question for this text, without refering to the texts contents directly: {document} <start_of_turn>model Sure, here is the question:"
    question = generate(llm_model, tokenizer, prompt=prompt, verbose=False, max_tokens=1000)
    question = question.replace("\n", "")
    question = question.replace("*", "")
    questions.append((id, question, embeddings))

Generating questions: 100%|██████████| 121/121 [02:40<00:00,  1.33s/it]


In [6]:
import random
random_questions = random.sample(questions, 5)
for question in random_questions:
    print(question[1])

What is the purpose of the copyright notice and permission notice included in the text?
How can pairs of data be sorted by their second element using lambda expressions?
What is the purpose of the default argument values in the function `ask_ok`?
What does the text following the heading "4. More Control Flow Tools Python Tutorial, Release 3.7.0" suggest about the content of the chapter?
What is the purpose of the equal sign (=) in the text?


In [7]:
import torch
import numpy as np
from torch.nn.functional import cosine_similarity

class VectorStore:
    def __init__(self, vectors, ids, device):
        """
        Initializes the store with vectors and their corresponding ids.
        
        :param vectors: A list of numpy arrays.
        :param ids: A list of ids corresponding to the vectors.
        """
        assert len(vectors) == len(ids), "Vectors and IDs must have the same length"
        
        # Convert numpy arrays to PyTorch tensors and store them
        self.vectors = [torch.tensor(vec, dtype=torch.float32, device=device) for vec in vectors]
        print(len(self.vectors))
        self.ids = ids

    def find_closest_id(self, vector):
        """
        Finds the ID of the stored vector closest to the given vector based on cosine similarity.
        
        :param vector: A numpy array representing the vector to compare.
        :return: The id of the closest vector.
        """
        
        # Calculate cosine similarity between the given vector and each stored vector
        similarities = [cosine_similarity(vector.unsqueeze(0), vec.unsqueeze(0)) for vec in self.vectors]
        
        # Find the index of the maximum similarity
        max_index = torch.argmax(torch.tensor(similarities)).item()
        
        # Return the corresponding ID
        return self.ids[max_index], self.vectors[max_index]

In [8]:
vectors = [vector for id, question, vector in questions]
ids = [id for id, question, vector in questions]
vector_store = VectorStore(vectors, ids, 'mps')

121


In [9]:
def batch_to_device(batch, target_device: torch.device):
    """
    send a pytorch batch to a device (CPU/GPU)
    """
    for key in batch:
        if isinstance(batch[key], torch.Tensor):
            batch[key] = batch[key].to(target_device)
    return batch

In [10]:
import random
import copy

model_id = 'thenlper/gte-large'
model = SentenceTransformer(model_id)
optimizer = optim.Adam(model.parameters(), lr=1e-5, eps=1e-5)

model.train()
loss_fn = nn.CosineEmbeddingLoss()

episodes = 200
batch_size = 16
num_batches = 5

for episode in range(episodes):
    predictions = []
    labels = []
    accuracies = []
    ep_reward = 0

    for _ in tqdm(range(num_batches), desc=f"Episode {episode+1}/{episodes}"):
        batch = random.sample(questions, batch_size)
        
        for id, question, vector in batch:
            tokenized_question = model.tokenize([question])
            tokenized_question = batch_to_device(tokenized_question, model.device)
            embedding = model(tokenized_question)['sentence_embedding'][0]
            prediction, label_vector = vector_store.find_closest_id(embedding)
            if prediction == id:
                reward = 1
            else:
                reward = 0
            predictions.append(embedding)
            labels.append(label_vector)
            accuracies.append(reward)
    
    predictions = torch.stack(predictions)
    label_vector = torch.stack(labels)
    optimizer.zero_grad()
    loss = loss_fn(predictions, label_vector, torch.ones(batch_size * num_batches).to(model.device))
    loss.backward()
    optimizer.step()
    
    accuracy = sum(accuracies) / len(accuracies)
    print(f"Episode {episode+1}/{episodes} loss: {loss.item()} accuracy: {accuracy}")


Episode 1/200: 100%|██████████| 5/5 [00:06<00:00,  1.34s/it]


Episode 1/200 loss: 0.09957718849182129 accuracy: 0.8


Episode 2/200: 100%|██████████| 5/5 [00:07<00:00,  1.40s/it]


Episode 2/200 loss: 0.0974433645606041 accuracy: 0.75


Episode 3/200: 100%|██████████| 5/5 [00:07<00:00,  1.41s/it]


...

Episode 198/200 loss: 0.026861492544412613 accuracy: 0.725


Episode 199/200: 100%|██████████| 5/5 [00:06<00:00,  1.29s/it]


Episode 199/200 loss: 0.025587117299437523 accuracy: 0.6625


Episode 200/200: 100%|██████████| 5/5 [00:06<00:00,  1.31s/it]


Episode 200/200 loss: 0.0270228274166584 accuracy: 0.7


In [13]:
accuracies = {}
lengths = [1, 2, 3, 4, 5]
for n_results in lengths:
    datapoints = []
    for id, question, vector in questions:
        results = collection.query(query_texts=question, n_results=n_results)
        result_ids = results["ids"][0]
        if id in result_ids:
            datapoints.append(1)
        else:
            datapoints.append(0)

    accuracies[n_results] = sum(datapoints) / len(datapoints)
accuracies

ValueError: Expected embeddings to be a list, got [[-0.01044687 -0.02135091  0.01859619 ...  0.01973298 -0.00316574
  -0.01486639]]